In [1]:
from bruker.api.topspin import Topspin
from bruker.data.nmr  import *
import pandas as pd
import os as os
import numpy as np
import pprint
import time

In [2]:
top = Topspin()
dp = top.getDataProvider()
# First, get the data set currently shown in TopSpin
#
curdat = top.getDataProvider().getCurrentDataset()
if curdat is None or curdat.getDimension() > 1:
    err = 'This example requires 1D dataset to be opened in TopSpin'
    top.showError(err)
    raise Exception(err)
    
# apply some processing commands
curdat.launch('efp')

info = curdat.getInfo()
# print the keys of the dictionary first
# dict_keys(['dataset_path', 'raw_data_dimension', 'proc_data_dimension', 'acquisition_date', 'acquisition_nuclei', 'spectrometer_frequency', 'solvent', 'pulse_program', 'spec_typ', 'title'])
# pretty print of dictionary
pprint.pprint(info)

Exception: This example requires 1D dataset to be opened in TopSpin

In [13]:
#RUN THIS CELL FOR ONEPULSE spectrum
#read the whole processed spectrum into dictionary
#dict_keys(['indexRanges', 'physicalRanges', 'dataPoints'])

title = '18.1nofiller-60C'

output_file_path = rf'C:\Users\felix\NMRdeconvoluted_txt\18.1nofiller\{title}.txt'
#################set title############################

curdat = top.getDataProvider().getCurrentDataset()
specData = curdat.getSpecDataPoints()

curdat.launch('abs')

if specData.get(EXCEPTION):
    print('Error :',specData.get(EXCEPTION).details())
    exit(-1)

# gives me my ranges
ir = specData['indexRanges'][0]
pr = specData['physicalRanges'][0]
print(pr)
print(ir)

start_point = pr.getStart()
end_point = pr.getEnd()
num_points = ir.getNum()

#convert spectral data to a format that decon3.0 can use
recovered = specData['dataPoints']

#impletement startpoint and endpoint into heading of the textfile before saving
heading = f"Start Point: {start_point}\nEnd Point: {end_point}\n"

# Save the data points to a text file with heading
np.savetxt(output_file_path, recovered, header=heading)


{'start': 393.4158020019531, 'end': -463.89411974545703, 'unit': 'ppm', 'referenceFrequency': 0.0}
{'firstPoint': 0, 'numberOfPoints': 16384}


In [26]:
# RUN THIS CELL FOR For satrec1nodec T1
# Open the satrec1nodec file in Bruker and adjust phase manually
def create_folder(filepath):
    try:
        # Use os.makedirs() to create the folder and any necessary parent folders
        os.makedirs(filepath)
        print(f"Folder created at {filepath}")
    except FileExistsError:
        print(f"Folder already exists at {filepath}")

#################################################################set title################################################################

foldertitle= '18.1nofiller60C'

firstfolder= '18.1nofiller'
    
#get current data shown
curdat = top.getDataProvider().getCurrentDataset()
specData = curdat.getSpecDataPoints()

#baseline correction
curdat.launch('abs')
#show title to make sure its the right dataset
info = curdat.getInfo()
print(info['title'])

if specData.get(EXCEPTION):
    print('Error :',specData.get(EXCEPTION).details())
    exit(-1)

# gives me my ranges
ir = specData['indexRanges'][0]
pr = specData['physicalRanges'][0]
print(pr)
print(ir)

start_point = pr.getStart()
end_point = pr.getEnd()
num_points = ir.getNum()

#convert spectral data to a format that decon3.0 can use
recovered = specData['dataPoints']

#Make folder
folder_path = rf'C:\Users\felix\NMRdeconvoluted_txt\{firstfolder}\{foldertitle}'
create_folder(folder_path)

#for loop that goes through the vdlist and makes a text file for each measurement
for i in range(20):
    #write the big file into multiple small ones define slicing index
    str_index = i*num_points
    end_index = (i+1)*num_points
    #get the i-th num_points from recovered array slicing the array
    snippet = recovered[str_index:end_index:1]
    #impletement startpoint and endpoint into heading of the textfile before saving
    heading = f"Start Point: {start_point}\nEnd Point: {end_point}\n"

    #save it in a text
    output_file_path = rf'C:\Users\felix\NMRdeconvoluted_txt\{firstfolder}\{foldertitle}\{i+1}.txt'
    np.savetxt(output_file_path, snippet, header=heading)


T=55C (328K)
7Li Minion static
Sample = PEO with LiTFSI 18-1 (no filler)
in sec
0.0001
0.0002
0.0005
0.001
0.002
0.005
0.01
0.02
0.03
0.05
0.1
0.15
0.2
0.3
0.5
1
2
3
5
10
{'start': 502.9158935546875, 'end': -499.7857693077858, 'unit': 'ppm', 'referenceFrequency': 0.0}
{'firstPoint': 0, 'numberOfPoints': 32768}
Folder created at C:\Users\felix\NMRdeconvoluted_txt\18.1nofiller\18.1nofiller55C
